# Read and Copy the dimensions


In [19]:
import numpy as np
import netCDF4
import os

NEWFILE_PATH = 'newfiletest.nc'

WRFOUT_FILE_PATH = 'wrfout_d01_2017-09-09_12:00:00'
#Open original for reading and new diagnostic for writing
orig_dataset = netCDF4.Dataset(WRFOUT_FILE_PATH, 'r')

#if newfile already exits, remove it to avoid potential
#permission problems
if os.path.isfile(NEWFILE_PATH):
    os.remove(NEWFILE_PATH)
diag_dataset = netCDF4.Dataset(NEWFILE_PATH, 'w', clobber=True)

# List of all dimensions from original dataset
# This is just for show
dimkeys = orig_dataset.dimensions.keys()
print('dimkeys: ' + str(dimkeys))
for the_key in orig_dataset.dimensions.keys():
    print('key: ' + str(the_key) +
         ', name: ' + str(orig_dataset.dimensions[the_key].name) + 
         ', value: ' + str(orig_dataset.dimensions[the_key].size))

# Write the same dimension to the diagnostic dataset
for the_key in orig_dataset.dimensions.keys():
    the_value = orig_dataset.dimensions[the_key].size
    the_name = orig_dataset.dimensions[the_key].name
    diag_dataset.createDimension(the_name, the_value)
    print(str(the_key) + str(the_value))

# List all global attributes and their values
#This is just for show
global_attributes = orig_dataset.ncattrs()
for attr in global_attributes:
    print(attr + ': ' + str(getattr(orig_dataset, attr)))
    
# Write the global attributes to diagnostic file
for attr in orig_dataset.ncattrs():
    attr_value = getattr(orig_dataset, attr)
    setattr(diag_dataset, attr, attr_value)

# orig_dataset.close
# diag_dataset.close  

dimkeys: odict_keys(['Time', 'DateStrLen', 'west_east', 'south_north', 'bottom_top', 'bottom_top_stag', 'soil_layers_stag', 'west_east_stag', 'south_north_stag'])
key: Time, name: Time, value: 1
key: DateStrLen, name: DateStrLen, value: 19
key: west_east, name: west_east, value: 97
key: south_north, name: south_north, value: 69
key: bottom_top, name: bottom_top, value: 50
key: bottom_top_stag, name: bottom_top_stag, value: 51
key: soil_layers_stag, name: soil_layers_stag, value: 4
key: west_east_stag, name: west_east_stag, value: 98
key: south_north_stag, name: south_north_stag, value: 70
Time1
DateStrLen19
west_east97
south_north69
bottom_top50
bottom_top_stag51
soil_layers_stag4
west_east_stag98
south_north_stag70
TITLE:  OUTPUT FROM WRF V3.9.1.1 MODEL
START_DATE: 2017-09-07_12:00:00
SIMULATION_START_DATE: 2017-09-07_12:00:00
WEST-EAST_GRID_DIMENSION: 98
SOUTH-NORTH_GRID_DIMENSION: 70
BOTTOM-TOP_GRID_DIMENSION: 51
DX: 30000.0
DY: 30000.0
SKEBS_ON: 0
SPEC_BDY_FINAL_MU: 1
USE_Q_DIABATI

In [20]:
# Create pressure height and temperature fields and write to new file

# Let's get the P and PB arrays
P = orig_dataset.variables['P'][:]
PB = orig_dataset.variables['PB'][:]

# Create the total pressure millibar field
P_mb = (P + PB) * 0.01

# Write it, along with attributes, to the diagnostic field
pmb = diag_dataset.createVariable('P_mb', np.float32, 
                                 ('Time', 'bottom_top',
                                  'south_north', 'west_east'))
pmb.units = "mb"
pmb.long_name = "Pressure"
pmb[:] = P_mb